<a href="https://colab.research.google.com/github/basiaChen2000/Assembler/blob/main/112%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A_part1_%E8%B3%87%E6%96%99%E5%89%8D%E8%99%95%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 112機器學習期末報告_part1_資料前處理
# 2023/12/15
# 7111029062 陳思蓓

- 資料前處理讀取檔案 BankChurners.csv
- 處理後匯出檔案 processed_data.csv

In [ ]:
import pandas as pd
data = pd.read_csv('BankChurners.csv', delimiter=",", header='infer')

### 1. 刪除ID欄位

In [ ]:
# 刪除 CLIENTNUM 特徵
data.drop('CLIENTNUM', axis=1, inplace=True)

### 2. 確認資料集中沒有missing value


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  object 
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

### 3. 資料正規劃，做 Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 定義需要簡化的特徵
features_to_simplify = ['Education_Level', 'Marital_Status', 'Card_Category', 'Gender']

# 對具特徵進行簡化
for feature in features_to_simplify:
    label_encoder = LabelEncoder()
    data[feature] = label_encoder.fit_transform(data[feature])

# 將 Attrition_Flag 改為 0 和 1
attrition_mapping = {'Existing Customer': 0, 'Attrited Customer': 1}
data['Attrition_Flag'] = data['Attrition_Flag'].map(attrition_mapping)

# Income_Category 另外處理（有順序性）
income_mapping = {
    'Less than $40K': 0,
    '$40K - $60K': 1,
    '$60K - $80K': 2,
    '$80K - $120K': 3,
    '$120K +': 4,
    'Unknown': 5  # 或者你可以將 Unknown 視為一個單獨的分類
}
data['Income_Category'] = data['Income_Category'].map(income_mapping)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  int64  
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  int64  
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  int64  
 5   Marital_Status            10127 non-null  int64  
 6   Income_Category           10127 non-null  int64  
 7   Card_Category             10127 non-null  int64  
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

### 4. 特徵選擇
- 4.1 刪除相關程度高的欄位
  - Avg_Open_To_Buy 與 Credit_Limit 相關程度是 1
  - 刪除 Avg_Open_To_Buy
- 4.2 Variance Thresholding
- 4.3 PCA

In [ ]:
# # 刪除相關程度高的欄位
# import seaborn as sns
# import matplotlib.pyplot as plt

# # 設定 Seaborn 風格
# sns.set(style="white")

# # 計算相關性矩陣
# correlation_matrix = data.corr()

# # 繪製熱圖
# plt.figure(figsize=(12, 10))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
# plt.title("Correlation Matrix Heatmap")
# plt.show()


In [ ]:
data.drop('Avg_Open_To_Buy', axis=1, inplace=True)

4.2 Variance Thresholding

In [ ]:
# # Variance Thresholding
# from sklearn.feature_selection import VarianceThreshold

# # 創建 VarianceThreshold 實例，指定閾值
# threshold = 0.6  # 根據實際情況調整閾值
# selector = VarianceThreshold(threshold)

# # 將特徵選擇應用到數據
# data_selected = selector.fit_transform(data)

# # 獲取被選中的特徵索引
# selected_features = data.columns[selector.get_support()]

# # 將選擇後的數據轉換為數據框
# data_selected = pd.DataFrame(data_selected, columns=selected_features)

# # 輸出處理後的數據信息
# data_selected['Attrition_Flag'] = data['Attrition_Flag']
# data = data_selected
# data.info()

In [ ]:
# data_selected.info()

- 列出分佈

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # 繪製每個特徵的直方圖
# num_features = len(data.columns)
# num_rows = 2
# num_cols = (num_features + num_rows - 1) // num_rows

# plt.figure(figsize=(16, 8))

# for i, feature in enumerate(data.columns):
#     plt.subplot(num_rows, num_cols, i + 1)  # 調整子圖的位置
#     sns.histplot(data[feature], kde=True, bins=20)
#     plt.title(feature)

# # 處理多餘的子圖
# for j in range(i + 1, num_rows * num_cols):
#     plt.delaxes(plt.subplot(num_rows, num_cols, j + 1))

# plt.tight_layout()
# plt.show()

### 5. 處理離群值
- 四分位距
  - lower_bound = Q1 - 1.5 * IQR
  - upper_bound = Q3 + 1.5 * IQR

In [ ]:
data_remember = data['Attrition_Flag']

import numpy as np
# 計算每個欄位的四分位數
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)

# 計算每個數值型欄位的四分位距
IQR = Q3 - Q1

# 計算每個數值型欄位的上下限
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


numeric_columns = data.select_dtypes(include=np.number).columns

# 迴圈遍歷每個數值型欄位，替換離群值
for col in numeric_columns:
    data[col] = data[col].clip(lower=lower_bound[col], upper=upper_bound[col])

data['Attrition_Flag'] = data_remember
print("DataFrame without Outliers:")
print(data)

DataFrame without Outliers:
       Attrition_Flag  Customer_Age  Gender  Dependent_count  Education_Level  \
0                   0          45.0       1                3                3   
1                   0          49.0       0                5                2   
2                   0          51.0       1                3                2   
3                   0          40.0       0                4                3   
4                   0          40.0       1                3                5   
...               ...           ...     ...              ...              ...   
10122               0          50.0       1                2                2   
10123               1          41.0       1                2                6   
10124               1          44.0       0                1                3   
10125               1          30.0       1                2                2   
10126               1          43.0       0                2                2   


### 6. 資料標準化，使用Min-Max

In [ ]:
# 假設 'Attrition_Flag' 是目標變數的列名
unique_values = data['Attrition_Flag'].unique()

print("Unique values in Attrition_Flag:", unique_values)

Unique values in Attrition_Flag: [0 1]


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 創建一個 MinMaxScaler
scaler = MinMaxScaler()

# 對整個數據框中的所有數值型特徵進行最小-最大標準化
data[data.select_dtypes(include='number').columns] = scaler.fit_transform(data.select_dtypes(include='number'))


### 6. PCA

In [ ]:
# import pandas as pd
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler

# # 假設你的數據已經經過前處理，並存儲在名為 data 的 DataFrame 中

# # 指定要排除的欄位
# exclude_columns = ['Customer_Age', 'Gender', 'Dependent_count', 'Education_Level', 'Marital_Status',
#                    'Income_Category', 'Card_Category']

# # 提取需要進行 PCA 的欄位
# selected_columns = ['Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
#                     'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
#                     'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct',
#                     'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']

# # 提取特徵和目標變數
# X = data[selected_columns]
# y = data['Attrition_Flag']

# # 標準化特徵
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # 創建 PCA 實例
# pca = PCA(n_components=5)

# # 將數據轉換為主成分
# X_pca = pca.fit_transform(X_scaled)

# # 將 PCA 轉換後的數據轉換為 DataFrame
# columns_pca = [f'PC{i+1}' for i in range(X_pca.shape[1])]
# X_pca_df = pd.DataFrame(data=X_pca, columns=columns_pca)

# # 將排除的欄位值加回去
# for col in exclude_columns:
#     X_pca_df[col] = data[col]
# X_pca_df['Attrition_Flag'] = y
# # 顯示新的 DataFrame 的信息
# X_pca_df.info()

# data = X_pca_df

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 假設你的數據已經經過前處理，並存儲在名為 data 的 DataFrame 中

# 提取特徵和目標變數
X = data.drop('Attrition_Flag', axis=1)
y = data['Attrition_Flag']

# 標準化特徵
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 創建 PCA 實例
pca = PCA(n_components= 17)

# 將數據轉換為主成分
X_pca = pca.fit_transform(X_scaled)

# 將 PCA 轉換後的數據轉換為 DataFrame
columns_pca = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=columns_pca)

# 將目標變數添加回 DataFrame
X_pca_df['Attrition_Flag'] = y
X_pca_df.info()
data = X_pca_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PC1             10127 non-null  float64
 1   PC2             10127 non-null  float64
 2   PC3             10127 non-null  float64
 3   PC4             10127 non-null  float64
 4   PC5             10127 non-null  float64
 5   PC6             10127 non-null  float64
 6   PC7             10127 non-null  float64
 7   PC8             10127 non-null  float64
 8   PC9             10127 non-null  float64
 9   PC10            10127 non-null  float64
 10  PC11            10127 non-null  float64
 11  PC12            10127 non-null  float64
 12  PC13            10127 non-null  float64
 13  PC14            10127 non-null  float64
 14  PC15            10127 non-null  float64
 15  PC16            10127 non-null  float64
 16  PC17            10127 non-null  float64
 17  Attrition_Flag  10127 non-null 

### 7. SMOTE解決資料不平衡
- 對訓練集做

In [ ]:
# from imblearn.over_sampling import SMOTE
# from sklearn.model_selection import train_test_split
# import pandas as pd

# # 提取特徵和目標變數
# X = data.drop('Attrition_Flag', axis=1)
# y = data['Attrition_Flag']

# # 將數據集分為訓練集和測試集
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 使用SMOTE生成合成樣本，以實現2:1的比例
# smote = SMOTE(sampling_strategy=0.4, random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# # 將生成的樣本添加到原始數據中
# data_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns),
#                             pd.DataFrame(y_resampled, columns=['Attrition_Flag'])], axis=1)

# # 顯示生成的樣本數量
# print("Original class distribution:", pd.Series(y_train).value_counts())
# print("Resampled class distribution:", pd.Series(y_resampled).value_counts())
# data = data_resampled

### 匯出檔案 processed_data.csv

In [ ]:
# 存儲編碼後的數據為新檔案
data.to_csv('processed_data_pca17.csv', index=False)


#### 檢查一下測試與訓練資料 ＾＾b

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 假設你的特徵和標籤是這樣的
X = data.drop('Attrition_Flag', axis=1)
y = data['Attrition_Flag']

# 分割資料集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 計算訓練集中標籤的分佈
train_label_distribution = y_train.value_counts(normalize=True)
print("Training Set Label Distribution:")
print(train_label_distribution)

# 計算測試集中標籤的分佈
test_label_distribution = y_test.value_counts(normalize=True)
print("\nTesting Set Label Distribution:")
print(test_label_distribution)

Training Set Label Distribution:
0.0    0.839526
1.0    0.160474
Name: Attrition_Flag, dtype: float64

Testing Set Label Distribution:
0.0    0.838598
1.0    0.161402
Name: Attrition_Flag, dtype: float64


## 接下來
- 112機器學習期末報告_part2_Logistic Regression.ipynb
- part 2 : https://colab.research.google.com/drive/1WdrRpgr_f0QCgkFmgY91kSz2LUjYPNzc


## 其他part

- part 2 - Logistic Regression : https://colab.research.google.com/drive/1WdrRpgr_f0QCgkFmgY91kSz2LUjYPNzc
- part 3 - Bernoulli Naive Bayes : https://colab.research.google.com/drive/10qsCAieQL3K3DCCCUike05G2tj0qMeH_#scrollTo=62wX6d9tx9Dm
- part 4 - Gaussian Naive Bayes: https://colab.research.google.com/drive/1RneJrq6lHWHpoQalw5PqfRJQRATleMKP#scrollTo=GKZ-6UFJzMk2
- part 5 - SVM : https://colab.research.google.com/drive/13lZfMt8wvlpBsFScI-UfyCG0kJ_UwUiu#scrollTo=y-AZi4GU4ENY
- part 6 - Voting classifier : https://colab.research.google.com/drive/18Lc74nzb74ZVEgIJ2-n-1y4eM4gV2ASl#scrollTo=4ARuZiQ32ypO
- part 7 - Bagging classifier : https://colab.research.google.com/drive/1BjH0kDM4h7eAEQzYgJx5I9tf43uJNHAg#scrollTo=JeOMwpIY37XC
- other : https://colab.research.google.com/drive/1n_3NtKsvYMpZR_verOCk8kHRaIEzch7g?hl=zh-tw#scrollTo=yDtmaLf_3rP7

Feature Importance Filter

In [ ]:
# X=no.fit_transform(boston.data) ##useNormalizer
# n Y=boston.target
# n fromsklearn.feature_selectionimportSelectKBest,f_regression n kb_regr = SelectKBest(f_regression) ## 預設是選 10 個
# n X_b=kb_regr.fit_transform(X,Y)
# n X_b.shape
# n kb_regr.scores_